In [2]:
from simple_salesforce import Salesforce
from functools import lru_cache

import logging
import os
import pandas as pd

logging.basicConfig(level=logging.DEBUG)


sf_client = Salesforce(
    username=os.getenv("SF_USERNAME"),
    password=os.getenv("SF_PASSWORD"),
    security_token=os.getenv("SF_TOKEN")
)

resp = sf_client.describe()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): login.salesforce.com:443
DEBUG:urllib3.connectionpool:https://login.salesforce.com:443 "POST /services/Soap/u/57.0 HTTP/1.1" 200 850
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): creative-moose-lqi7kc-dev-ed.trailblaze.my.salesforce.com:443
DEBUG:urllib3.connectionpool:https://creative-moose-lqi7kc-dev-ed.trailblaze.my.salesforce.com:443 "GET /services/data/v57.0/sobjects HTTP/1.1" 200 None


In [7]:
@lru_cache(maxsize=100)
def get_object_metadata(sf_client, object_name):
    return getattr(sf_client, object_name).describe()

In [8]:
resp = get_object_metadata(sf_client, 'Property__c')

DEBUG:urllib3.connectionpool:https://creative-moose-lqi7kc-dev-ed.trailblaze.my.salesforce.com:443 "GET /services/data/v57.0/sobjects/Property__c/describe HTTP/1.1" 200 None


In [9]:
resp.keys()

odict_keys(['actionOverrides', 'activateable', 'associateEntityType', 'associateParentEntity', 'childRelationships', 'compactLayoutable', 'createable', 'custom', 'customSetting', 'deepCloneable', 'defaultImplementation', 'deletable', 'deprecatedAndHidden', 'extendedBy', 'extendsInterfaces', 'feedEnabled', 'fields', 'hasSubtypes', 'implementedBy', 'implementsInterfaces', 'isInterface', 'isSubtype', 'keyPrefix', 'label', 'labelPlural', 'layoutable', 'listviewable', 'lookupLayoutable', 'mergeable', 'mruEnabled', 'name', 'namedLayoutInfos', 'networkScopeFieldName', 'queryable', 'recordTypeInfos', 'replicateable', 'retrieveable', 'searchLayoutable', 'searchable', 'sobjectDescribeOption', 'supportedScopes', 'triggerable', 'undeletable', 'updateable', 'urls'])

In [10]:
resp2 = get_object_metadata(sf_client, 'Property__c')

In [6]:
resp.keys()

odict_keys(['encoding', 'maxBatchSize', 'sobjects'])

In [10]:
sobjects = resp['sobjects']

In [11]:
len(sobjects[0].keys())

28

In [12]:
dict_ = {}
for item in sobjects:
    if dict_.get(len(item)) is None:
        dict_[len(item)] = 1
    else:
        dict_[len(item)] += 1

In [13]:
dict_

{28: 891}

In [14]:
df = pd.DataFrame.from_records(sobjects)

In [15]:
df[df.custom].dtypes

activateable               bool
associateEntityType      object
associateParentEntity    object
createable                 bool
custom                     bool
customSetting              bool
deepCloneable              bool
deletable                  bool
deprecatedAndHidden        bool
feedEnabled                bool
hasSubtypes                bool
isInterface                bool
isSubtype                  bool
keyPrefix                object
label                    object
labelPlural              object
layoutable                 bool
mergeable                  bool
mruEnabled                 bool
name                     object
queryable                  bool
replicateable              bool
retrieveable               bool
searchable                 bool
triggerable                bool
undeletable                bool
updateable                 bool
urls                     object
dtype: object

In [16]:
df.to_csv("../data/sobject_property_inventory.csv")

In [31]:
property = getattr(sf_client, 'Property__c').describe()

In [32]:
property_df = pd.DataFrame.from_records(property['fields'])
property_df.to_csv("../data/property__c_field_inventory.csv")

In [33]:
property_df[property_df.type == 'picklist'].dtypes

aggregatable                      bool
aiPredictionField                 bool
autoNumber                        bool
byteLength                       int64
calculated                        bool
calculatedFormula               object
cascadeDelete                     bool
caseSensitive                     bool
compoundFieldName               object
controllerName                  object
createable                        bool
custom                            bool
defaultValue                    object
defaultValueFormula             object
defaultedOnCreate                 bool
dependentPicklist                 bool
deprecatedAndHidden               bool
digits                           int64
displayLocationInDecimal          bool
encrypted                         bool
externalId                        bool
extraTypeInfo                   object
filterable                        bool
filteredLookupInfo              object
formulaTreatNullNumberAsZero      bool
groupable                

In [36]:
property_df.name.unique()

array(['Id', 'OwnerId', 'IsDeleted', 'Name', 'CreatedDate', 'CreatedById',
       'LastModifiedDate', 'LastModifiedById', 'SystemModstamp',
       'LastViewedDate', 'LastReferencedDate', 'Price__c', 'Agent_ID__c',
       'Street_Address__c', 'Current_Status__c',
       'Unique_Street_Adddress__c'], dtype=object)

In [39]:
property_df[property_df.name == 'Unique_Street_Adddress__c'].T

,15
aggregatable,True
aiPredictionField,False
autoNumber,False
byteLength,300
calculated,False
calculatedFormula,None
cascadeDelete,False
caseSensitive,False
compoundFieldName,None
controllerName,None


In [34]:
property_df[property_df.type == 'text'].picklistValues.iloc[0]

IndexError: single positional indexer is out-of-bounds

methods - get fields for object, get properties for field

for each object
check for expected columns


for each field
if picklist, check that all values are valid
if not nullable, check that values are not null
if unique, check that values are unique


if it's 

all we're doing is creating a gx context. we'll figure out how to use multiple contexts in the future. for now, we can namespace them under contexts/object_name

